In [1]:
import argparse
import os, sys
from utils.data_utils import *
from glob import glob
import pandas as pd
import numpy as np
import shutil
from tqdm import tqdm
%matplotlib inline  
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [12,9]
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.colheader_justify', 'left')

In [2]:
def get_data_dir(a, t):
    if t:
        if a:
            data_dir = 'train_ex'
        else:
            data_dir = 'train'
    else:
        if a:
            data_dir = 'valid_ex'
        else:
            data_dir = 'valid'
    return data_dir

def load_loss_df(l_dirs, augmented=False, train=False):
    data_dir = get_data_dir(augmented, train)
    dataframes = []
    for d in l_dirs:
        dataframes.append(pd.read_csv(os.path.join('predictions', data_dir, d, 'models.csv')))

    df_l = pd.concat(dataframes, ignore_index=True)
    return df_l

def get_gt_name(a, t):
    if t:
        if a:
            name = 'Y_train_augmented.npz'
        else:
            name = 'Y_train.npz'
    else:
        if a:
            name = 'Y_valid_augmented.npz'
        else:
            name = 'Y_valid.npz'
    return name

def load_ground_truth(data_dir, augmented=False, train=False):
    gt_name = get_gt_name(augmented, train)
    y_true_path = os.path.join('augmented', data_dir, gt_name)
    y_true = npz_to_ndarray(np.load(y_true_path))
    return y_true

def load_preds(df_loss):
    preds = []
    for npz_path in tqdm(df_loss.res_path):
        p = npz_to_ndarray(np.load(npz_path))
        preds.append(p)
    return np.array(preds)

def load_test_df(l_dirs):
    dataframes = []
    for d in l_dirs:
        dataframes.append(pd.read_csv(os.path.join('test_predictions', d, 'models.csv')))

    df_l = pd.concat(dataframes, ignore_index=True)
    return df_l

def get_labels():
    labels = [
        'slash_burn',
        'clear',
        'blooming',
        'primary',
        'cloudy',
        'conventional_mine',
        'water',
        'haze',
        'cultivation',
        'partly_cloudy',
        'artisinal_mine',
        'habitation',
        'bare_ground',
        'blow_down',
        'agriculture',
        'road',
        'selective_logging'
    ]

    l_map = {l: i for i, l in enumerate(labels)}
    inv_map = {i: l for l, i in l_map.items()}
    return labels, l_map, inv_map

labels, label_map, inv_label_map = get_labels()

def to_tagging(one_hot_data, inv_label_map):
    res = pd.DataFrame(index=range(len(one_hot_data)), columns=['tags'])
    for j in tqdm(range(len(one_hot_data))):
        tags = []
        for i in range(17):
            if one_hot_data[j][i] == 1:
                tags.append(inv_label_map[i])
        res['tags'][j] = ' '.join(sorted(tags))
    return res

def get_test_df():
    inputs_dir = 'inputs'
    df_submission = pd.read_csv(os.path.join(inputs_dir, 'sample_submission_v2.csv'))
    return df_submission

def create_submission(pred, sufix):
    print("Reading submission dataframe")
    df_submission = get_test_df()
    print("Creating tags")
    p_tags = to_tagging(pred, inv_label_map)
    df_submission.tags = p_tags.tags
    submission_dir = os.path.join('submissions', 'ensemble_avg_test')
    if not os.path.exists(submission_dir):
        os.makedirs(submission_dir)
        
    file_name = 'submission_' + sufix + '.csv'
    print("Saving submission to file")
    df_submission.to_csv(os.path.join(submission_dir, file_name), index=False)
    print("%s created" % os.path.join(submission_dir, file_name))
    
def val_idx_to_test(val_idx, df_test, df_valid):
    res = [df_test[df_test.path == p].index[0] for p in df_valid.loc[val_idx].path]
    if len(res) != len(val_idx):
        missing_idx = -1
        for i in val_idx:
            if not df_valid.loc[i].path in list(df_test.path):
                missing_idx = i
        raise RuntimeError("Model %d not in the test dataframe" % missing_idx)
        
    return res

In [23]:
l_dirs = ['0.09']
df_valid = load_loss_df(l_dirs, augmented=True, train=False)
print("val dataframe size %d" % len(df_valid))
preds_valid = load_preds(df_valid)
print("val predictions shape %s" % str(preds_valid.shape))

  0%|          | 2/499 [00:00<00:28, 17.61it/s]

val dataframe size 499


100%|██████████| 499/499 [00:20<00:00, 24.34it/s]


val predictions shape (499, 64768, 17)


In [34]:
df_test = load_test_df(l_dirs)
print("test dataframe size %d" % len(df_test))
preds_test = load_preds(df_test)
print("val predictions shape %s" % str(preds_test.shape))

  6%|▌         | 3/50 [00:00<00:02, 22.57it/s]

test dataframe size 50


100%|██████████| 50/50 [00:01<00:00, 36.93it/s]

val predictions shape (50, 61191, 17)


In [6]:
y_true_valid = load_ground_truth('train_augmented_size_128_mult_8_seed_0', augmented=True, train=False)
print("Y ground truth shape %s" % str(y_true_valid.shape))

Y ground truth shape (64768, 17)


In [7]:
def vote(preds):
    return (preds.mean(axis=0) > 0.5).astype(int)

epsilon = 0.00001
def find_best_candidate(y_true, all_preds, ensemble):
    best_avg_score = 0
    best_candidate = None
    ensemble_preds = [all_preds[j] for j in ensemble]
    for i in tqdm(range(len(all_preds))):
        preds = ensemble_preds + [all_preds[i]]
        avg_pred = (np.mean(preds, axis=0) > 0.2).astype(int)
        avg_score = fbeta_score(y_true, avg_pred, beta=2, average='samples')
        if avg_score > best_avg_score:
            best_avg_score = avg_score                
            best_candidate = i          
                
    return best_candidate, best_avg_score

def optimize_score(y_true, preds, init_ensemble = [], exclude = []):
    best_ensemble = init_ensemble[:]
    best_score = 0
    scores = []
    counter = 0
    while True:
        best_candidate, ens_score = find_best_candidate(y_true, preds, ensemble=best_ensemble)
        sys.stdout.flush()
            
        if ens_score > (best_score + epsilon):
            best_ensemble.append(best_candidate)
            print("%d: Best candidate model is %d with improvement of %.5f" % (counter, best_candidate, ens_score-best_score))
            print("%d: Best ensemble %s" % (counter, best_ensemble))
            print("%d: New ensemble score %.5f" % (counter, ens_score))
            scores.append(ens_score)
            best_score = ens_score
            counter+=1
        else:
            print("Can't optimize any more")
            break
    
    return best_ensemble, scores

In [10]:
def optimize_with_split(n_splits, preds_in, y_true):
    preds = preds_in.copy()

    counter = 0
    bag_size = len(preds)/n_splits
    last_bag_size = len(preds) - bag_size*(n_splits-1)
    print("Split to bas of %d where last %d" % (bag_size, last_bag_size))

    all_ensembles = []
    all_scores = []
    orig_idxes = np.arange(len(preds))
    curr_idxes = orig_idxes.copy()
    while len(curr_idxes) > 0:
        print("Optimizing bag %d" % counter)
        selection = np.random.choice(curr_idxes, replace=False, \
                                     size=(bag_size if len(curr_idxes) > last_bag_size else last_bag_size))

        p = preds[selection]
        ensemble, scores = optimize_score(y_true, p)
        all_ensembles.append(selection[ensemble])
        all_scores.append(scores)
        curr_idxes = curr_idxes[[i not in selection for i in curr_idxes]]
        counter+=1

    best_scores = [slist[-1] for slist in all_scores]
    print(best_scores)
    print("Average score %.5f" % np.mean(best_scores))
    
    return all_ensembles, all_scores

In [18]:
# l = np.random.choice(np.arange(len(preds_valid)), size=32)
# preds_ = preds_valid[l].copy()
all_ensembles = []
all_scores = []
for i in range(3):
    print("\nIteration %d" % i)
    ensembles, scores = optimize_with_split(n_splits=3, preds_in=preds_valid, y_true=y_true_valid)
    all_ensembles += ensembles
    all_scores += scores


Iteration 0
Split to bas of 166 where last 166
Optimizing bag 0


  1%|          | 2/166 [00:00<00:11, 14.05it/s]

0: Best candidate model is 69 with improvement of 0.92802
0: Best ensemble [69]
0: New ensemble score 0.92802


  1%|          | 2/166 [00:00<00:12, 13.04it/s]

1: Best candidate model is 133 with improvement of 0.00189
1: Best ensemble [69, 133]
1: New ensemble score 0.92991


...

5: Best candidate model is 162 with improvement of 0.00056
5: Best ensemble [82, 114, 20, 0, 78, 162]
5: New ensemble score 0.93316


  1%|          | 1/166 [00:00<00:25,  6.59it/s]

6: Best candidate model is 118 with improvement of 0.00022
6: Best ensemble [82, 114, 20, 0, 78, 162, 118]
6: New ensemble score 0.93339


100%|██████████| 166/166 [00:14<00:00, 11.25it/s]

Can't optimize any more
[0.93253013180903299, 0.93301207022898069, 0.93338583616532222]
Average score 0.93298


In [31]:
for ens_list in all_ensembles:
    print(ens_list)

vote_preds = []
for ens_list in all_ensembles:
    ensemble_preds = [preds_valid[j] for j in ens_list]
    p = (np.mean(ensemble_preds, axis=0) > 0.2).astype(int)
    vote_preds.append(p)
    
final_pred = vote(np.array(vote_preds))
vote_score = fbeta_score(y_true_valid, final_pred, beta=2, average='samples')
print("Vote score on validation set %.5f" % vote_score)

[435 408  87 337 138 484 443 421 118 179 372]
[483 110 355 455 141 403 336 140]
[482 177 381 112 433 373 482 235 137 112 382 339 420 433 373 112]
[455 446 372 112 393 177  52 339 395]
[435 482 337 408  22 359 435]
[484 381 141 138  23 355 122 484 344 342 137 461]
[443 138 359 119 408 122]
[482 383 337 112 419 484 141 339 355 401]
[435 110 343 423 387 336 373  22 118 177  43 432 455 423]
Vote score on validation set 0.93348
